## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date
0,Port Alfred,-33.5906,26.8910,62.82,83,99,7.45,ZA,overcast clouds,2022-08-11 19:47:09
1,Lompoc,34.6391,-120.4579,78.94,74,40,14.97,US,mist,2022-08-11 19:47:11
2,Ushuaia,-54.8000,-68.3000,38.82,87,40,2.30,AR,scattered clouds,2022-08-11 19:43:07
3,Albany,42.6001,-73.9662,84.79,51,93,10.74,US,overcast clouds,2022-08-11 19:44:20
4,Port Alfred,-33.5906,26.8910,62.82,83,99,7.45,ZA,overcast clouds,2022-08-11 19:47:09


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 95
What is the maximum temperature you would like for your trip? 100


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_city_data_df = city_data_df.loc[(city_data_df['Max Temp']<=max_temp) & (city_data_df['Max Temp']>=min_temp)]

In [6]:
# 4a. Determine if there are any empty rows.
filtered_city_data_df[filtered_city_data_df['City'].isna()]

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date


In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
filtered_city_data_df.shape[0]

26

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = filtered_city_data_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
88,Cabo San Lucas,MX,95.61,few clouds,22.8909,-109.9124,
175,Cabo San Lucas,MX,95.61,few clouds,22.8909,-109.9124,
238,Elesbao Veloso,BR,95.85,scattered clouds,-6.2019,-42.1403,
243,Stephenville,US,98.92,scattered clouds,32.2207,-98.2023,
364,Abu Dhabi,AE,95.02,overcast clouds,24.4667,54.3667,
466,Bilma,NE,97.30,scattered clouds,18.6853,12.9164,
579,Pahrump,US,98.44,broken clouds,36.2083,-115.9839,
720,Cabo San Lucas,MX,95.61,broken clouds,22.8909,-109.9124,
750,Jacareacanga,BR,97.68,clear sky,-6.2222,-57.7528,
851,Rancho Palos Verdes,US,99.99,few clouds,33.7445,-118.3870,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df['Hotel Name']!=""]

In [11]:
# 8a. Create the output File (CSV)
output_data_file = './WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))